In [ ]:
import pandas as pd
from os.path import join, dirname, basename
import os
import numpy as np

from icecream import ic

In [ ]:
allExpmts_df = snakemake.params.allExpmts 
allRuns_don_df = snakemake.params.allRuns_don_df
out_fs = snakemake.output #.summary
# clone_dirs = []
# nuclear_dirs = []
# condition_pileup_dirs = []
# experiment_ids = []

In [ ]:
allRuns_don_df

## Get number of cells each donor

In [ ]:
#init_results_cond_df = pd.DataFrame(columns=["Donor", "# conditions"])
results_donor_df = allRuns_don_df.copy()
results_donor_df["# cells after demultiplex"] = -1

## Get multiplex ncells
for ind, val in allRuns_don_df.groupby("run"):
    print(ind)
    curr = pd.read_csv(join(val["indir"][0], "multiplex/cells_meta.tsv"), sep="\t", index_col=0)
    curr["donor"] = curr["donor"].astype("Int32")
    d_size = curr.groupby("donor").size()
    for (d, curr_size) in d_size.iteritems():
        results_donor_df.loc[f"{ind}_{d}", "# cells after demultiplex"] = curr_size
    #break

results_donor_df

#"mean clone size", "median clone size"

## Get number of cells each condition 

In [ ]:
results_cond_df = allExpmts_df.copy()
#pd.DataFrame(index=allExpmts_df.index, columns=["# donors", "Condition", 
results_cond_df["# cells nuclear open-chromatin QC"] = np.nan
results_cond_df["# variants that passed MT coverage QC and open-chromatin QC"] = np.nan
results_cond_df["# cells nuclear open-chromatin QC"] = np.nan

for ind, val in allExpmts_df.iterrows():
    print(ind)
    #cb = pd.read_csv(val["cb_f"], header=None)
    with open(val['cb_f'], 'r') as fp:
        x = len(fp.readlines())
    results_cond_df.loc[ind, "# cells nuclear open-chromatin QC"]  = x
    
    try:
        ncells, nvars = pd.read_csv(join(val["filt_dir"], "af_by_cell.tsv"),sep="\t", index_col=0).shape
        results_cond_df.loc[ind, "# cells that passed MT coverage QC and open-chromatin QC"]  = ncells
        results_cond_df.loc[ind, "# variants that passed MT coverage QC and open-chromatin QC"]  = nvars
    except FileNotFoundError:
        print(f"{ind} filt file not found")
results_cond_df

## Create condition-by-donor df

In [ ]:
def expand_df(ser, col="N_DONORS"):
    #print(ser["N_DONORS"])
    #print(len([ser]*ser["N_DONORS"]))
    #print(pd.DataFrame(ser))
    tmp = pd.concat([pd.DataFrame(ser).transpose()]*ser["N_DONORS"])
    tmp.index = [f"{ser.name}_{d}" for d in range(ser["N_DONORS"])]
    tmp["donor"] = [d for d in range(ser["N_DONORS"])]
    #print('tmp', tmp.index, tmp.columns)
    #ic(tmp.head())
    return tmp.to_dict()


applied_df = results_cond_df.apply(expand_df, axis=1)#, result_type="expand")
results_cond_don_df = pd.concat([pd.DataFrame(x[1]) for x in applied_df.items()], axis=0)


### Multiplex cells size
results_cond_don_df["ncells_demultiplex"] = np.nan
results_cond_don_df["ncells_demultiplex"] = results_cond_don_df["ncells_demultiplex"].astype('Int64')

#for ind, val in results_cond_don_df.groupby(["run"]):
#     curr_cells = pd.read_csv(join(val["indir"].values[0], f"anno_multiplex/gff_{val['gff'].values[0]}/se_cells_meta_labels.tsv"),
#                             sep="\t")
for ind, val in results_donor_df.iterrows():    
    curr_cells = pd.read_csv(join(val["indir"], f"anno_multiplex/gff_{val['gff']}/se_cells_meta_labels.tsv"),
                            sep="\t")
    curr_cells = curr_cells.reset_index().rename({"level_1":"cell"},axis=1).set_index("cell")
    curr_sz = curr_cells.groupby(["condition", "donor"]).size()
    for (cond, don), val2 in curr_sz.iteritems():
        print(cond, don)
        print(val2)
        if don != "None":
            nm = f"{val['run']}_{cond}_{don}"
            results_cond_don_df.loc[nm, "ncells_demultiplex"] = val2

## Save results

In [ ]:
results_donor_df = results_donor_df.drop(["indir", "anno_indir", "gff"], axis=1)
results_donor_df

In [ ]:
results_cond_df = results_cond_df.drop(["indir", "anno_indir", "cb_f", "filt_dir", "N_DONORS", "gff"], axis=1)
results_cond_df

In [ ]:
results_cond_don_df = results_cond_don_df.drop(["indir", "anno_indir", "cb_f", "filt_dir", "N_DONORS", "gff"], axis=1)

In [ ]:
results_donor_df.to_csv(out_fs["summary_don"])
results_cond_df.to_csv(out_fs["summary_cond"])
results_cond_don_df.to_csv(out_fs["summary_don_cond"])